# HomeWork 13

**DeadLine : 23:59 4 Day 1399**

**Total points : 100pts**

Import some prerequisites:

In [61]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
from tensorflow import keras
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
# example of creating a CNN with an efficient inception module
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers.merge import concatenate
from keras.utils import plot_model

Run this cell for plotting.

In [63]:
def plotter(img_list, r, w, gray, wr, hr, fig_name = None):
    '''
    Plots images' list with its' caption and saves result image if you want.

    Parameters:
        img_list (list): The list of tuples of image and its' caption.
        r (int): The number of row(s).
        w (int): The number of colunm(s).
        gray (bool): The flag for plotting images in grayscale mode.
        wr (int): The width of one figure.
        hr (int): The height of one figure.
        fig_name (str): The name of the image of the plot. if not set this parameter the plot doesn't save.
    '''
    
    plt.rcParams['figure.figsize'] = (wr, hr)
    for i in range(len(img_list)):
        plt.subplot(r, w, i + 1)
        if img_list[i][2] == 'img':
            if gray:
                plt.imshow(img_list[i][0], cmap = 'gray')
            else:
                plt.imshow(img_list[i][0])
            plt.xticks([])
            plt.yticks([])
        elif img_list[i][2] == 'hist':
            plt.bar(np.arange(len(img_list[i][0])), img_list[i][0], color = 'c')
        else:
            raise Exception("Only image or histogram. Use third parameter of tuples in img_list and set it to img or hist.")
        plt.title(img_list[i][1])
    if fig_name is not None:
        plt.savefig(fig_name + '.png')
    plt.show()

Run this cell for plotting evaluation.

In [64]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix


def evaluation(y_test, y_pred, fig_name, title):
    '''
    Prints accuracy and F1-score and plots confusion matrix and saves result image.

    Parameters:
        y_test (numpy.ndarray) : test labels (ground truth)
        y_pred (numpy.ndarray) : predicted labels 
        fig_name (str): The name of the image of the plot.
        title (str): The caption of the plot.
    '''

    acc = accuracy_score(y_test, y_pred)
    print('Accuracy for', title, '=', acc)
    f1 = f1_score(y_test, y_pred, average = 'macro')
    print('F1 score for', title, '=', f1)
    cm = confusion_matrix(y_test, y_pred)
    df_cm = pd.DataFrame(cm, range(10), range(10))
    plt.figure(figsize = (10, 7))
    sn.set(font_scale = 1.4) # for label size
    sn.heatmap(df_cm , annot = True, annot_kws = {"size" : 16}, fmt = 'g') # font size
    plt.title(title)
    plt.savefig(fig_name + '.png')
    plt.show()

Here is for your helper functions(optional)

In [65]:
# function for creating a projected inception module
def inception_module(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
	# 1x1 conv
	conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
	# 3x3 conv
	conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu')(layer_in)
	conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
	# 5x5 conv
	conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(layer_in)
	conv5 = Conv2D(f3_out, (3,3), padding='same', activation='relu')(conv5)
	conv5 = Conv2D(f3_out, (3,3), padding='same', activation='relu')(conv5)
 
  # conv5 = Conv2D(f3_out, (3,3), padding='same', activation='relu')(conv5)
  	# 3x3 max pooling
	pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
	pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
	# concatenate filters, assumes filters/channels last
	layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
	return layer_out

Getting the data:

In [66]:
from keras.datasets import fashion_mnist

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# plt.figure(figsize=(25,4))
# for i in range(16):
#   plt.subplot(1,16,i+1)
#   plt.imshow(x_train[i], cmap='gray')
#   plt.xticks([])
#   plt.yticks([])
#   plt.title(y_train[i])

num_classes = 10
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Create your model.

In [67]:
# define model with pooling
# model = keras.Sequential()
# model.add(keras.layers.Input(shape=x_train[0].shape))
# model.add(keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
# model.add(keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
# model.add(keras.layers.MaxPool2D())
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(units=num_classes, activation='softmax'))
# model.summary()

# define model input
visible = Input(shape=x_train[0].shape)

layer = Conv2D(32, (7,7), padding='same', activation='relu')(visible)
# add inception block 1
layer = inception_module(layer, 64, 96, 128, 16, 32, 32)
# add inception block 1
layer = inception_module(layer, 128, 128, 192, 32, 96, 64)

layer = Flatten()(layer)
# layer = Dense(units=1000, activation='softmax')(layer)
layer = Dense(units=num_classes, activation='softmax')(layer)


# create model
model = Model(inputs=visible, outputs=layer)
# summarize model
model.summary()


Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_141 (Conv2D)             (None, 28, 28, 32)   1600        input_15[0][0]                   
__________________________________________________________________________________________________
conv2d_145 (Conv2D)             (None, 28, 28, 16)   528         conv2d_141[0][0]                 
__________________________________________________________________________________________________
conv2d_143 (Conv2D)             (None, 28, 28, 96)   3168        conv2d_141[0][0]                 
____________________________________________________________________________________________

Train your model.

In [ ]:
 # compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train model
history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=10,
                    validation_data=(x_test, y_test),
                    shuffle=True)

# original_df = pd.DataFrame(history)
# original_df.to_pickle("content/trained.pkl")
model.save('content/drive/MyDrive/final_model.h5')

Epoch 1/10
600/600 [==============================] - 52s 85ms/step - loss: 18.2639 - accuracy: 0.7841 - val_loss: 0.3169 - val_accuracy: 0.8898
Epoch 2/10
123/600 [=====>........................] - ETA: 37s - loss: 0.2080 - accuracy: 0.9222

Evaluate your model

In [ ]:
# Suggestion: Using the evaluation function.
# Get y_pred for y_test from model.predict() then use evaluation(y_test, y_pred, fig_name, title)
# with proper name for fig_name and title.
temp = model.predict(x_test)
y_test = np.argmax(y_test, axis=1)
y_pred = np.argmax(temp, axis=1)
evaluation(y_test, y_pred, "result", "result")


Show 10 bad loss images.

In [ ]:
# Suggestion: Using the plotter function.
real_probs = []
for i in range(len(temp)):
  real_probs.append(temp[i][y_pred[i]])
min_probs = sorted(real_probs)[:10]
imgs = []
for i in range(len(min_probs)):
  imgs.append([cv2.cvtColor(x_test[real_probs.index(min_probs[i])], cv2.COLOR_BGR2RGB), '', 'img'])

plotter(imgs, 1, 10, False, 28, 28, fig_name = "10 worst")

End!!!